# Chatbot
In this tutorial, we'll be designing a chatbot with the capability to retain information from previous prompts and responses, enabling it to maintain context throughout the conversation. This ability sets it apart from LLMs, which typically process language in a more static manner.


---
## 1.&nbsp; Installations and Settings 🛠️
Let's replicate the process of downloading and installing the necessary libraries and model, just like we did in the previous LLM notebook. If you had any specific settings in the first notebook, such as using a CPU instead of a GPU, maintain those same configurations here.

Since we'll be continuing our work on Colab with a GPU, let's activate it by navigating to `Edit > Notebook Settings`. Select `GPU` and click `Save`.

In [ ]:
!pip3 install -qqq langchain --progress-bar off
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip3 install -qqq llama-cpp-python --progress-bar off

!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.1-GGUF mistral-7b-instruct-v0.1.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

  Installing build dependencies ... canceled
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 92, in resolve
    result = self._result = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 397, in resolve
    self._a

---
## 2.&nbsp; Setting up your LLM 🧠

In [1]:
from langchain.llms import LlamaCpp

llm = LlamaCpp(model_path = "/Users/sadiakhanrupa/Bootcamp Main Phase/Chapter_8_generative_Ai/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
               max_tokens = 2000,
               temperature = 0.1,
               top_p = 1,
               n_gpu_layers = -1)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/sadiakhanrupa/Bootcamp Main Phase/Chapter_8_generative_Ai/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32           

### 2.1.&nbsp; Test your LLM

In [2]:
answer_1 = llm.invoke("HOW SHOULD I LOVE MYSELF.")
print(answer_1)


llama_print_timings:        load time =    6552.10 ms
llama_print_timings:      sample time =       9.42 ms /   104 runs   (    0.09 ms per token, 11036.82 tokens per second)
llama_print_timings: prompt eval time =    6777.09 ms /    14 tokens (  484.08 ms per token,     2.07 tokens per second)
llama_print_timings:        eval time =    5305.27 ms /   103 runs   (   51.51 ms per token,    19.41 tokens per second)
llama_print_timings:       total time =   12219.11 ms /   117 tokens




I am a 25 year old female who has been struggling with my self esteem for as long as I can remember. I have always had a hard time loving myself and accepting myself for who I am. I have struggled with body image issues, anxiety, depression, and low self worth. I have tried therapy, medication, and various self help techniques but nothing seems to work. How do I learn to love myself?

Thank you in advance for any advice or guidance you may have.


---
## 3.&nbsp; Making a chatbot 💬
To transform a basic LLM into a chatbot, we'll need to infuse it with additional functionalities: prompts, memory, and chains.

**Prompts** are like the instructions you give the chatbot to tell it what to do. Whether you want it to write a poem, translate a language, or answer your questions. They provide the context and purpose for its responses.

**Memory** is like the chatbot's brain. It stores information from previous interactions, allowing it to remember what you've said and keep conversations flowing naturally.

The **chain** is like a road map that guides the conversation along the right path. It tells the LLM how to process your prompts, how to access the memory bank, and how to generate its responses.

In essence, prompts provide the direction, memory retains the context, and chains orchestrate the interactions.

In [3]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

### Prompt ###
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "Keep your answers very short, succinct, and to the point."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

### Memory ###
# We `return_messages=True` to fit into the MessagesPlaceholder
# `"chat_history"` must align with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key = "chat_history",
                                  return_messages=True)

### Chain ###
conversation = LLMChain(llm = llm,
                        prompt = prompt,
                        verbose = True,
                        memory = memory)

### 3.1.&nbsp; Test your chatbot

In [4]:
conversation({"question": "hi"})

/Users/sadiakhanrupa/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Llama.generate: prefix-match hit




> Entering new LLMChain chain...
Prompt after formatting:
System: Keep your answers very short, succinct, and to the point.
Human: hi



llama_print_timings:        load time =    6552.10 ms
llama_print_timings:      sample time =       1.97 ms /    13 runs   (    0.15 ms per token,  6592.29 tokens per second)
llama_print_timings: prompt eval time =    7029.50 ms /    21 tokens (  334.74 ms per token,     2.99 tokens per second)
llama_print_timings:        eval time =     636.62 ms /    12 runs   (   53.05 ms per token,    18.85 tokens per second)
llama_print_timings:       total time =    7700.71 ms /    33 tokens



> Finished chain.


{'question': 'hi',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='\nAI: Hello! How can I assist you today?')],
 'text': '\nAI: Hello! How can I assist you today?'}

In [5]:
conversation({"question": "How can I love my husband more"})



> Entering new LLMChain chain...
Prompt after formatting:
System: Keep your answers very short, succinct, and to the point.
Human: hi
AI: 
AI: Hello! How can I assist you today?
Human: How can I love my husband more


Llama.generate: prefix-match hit

llama_print_timings:        load time =    6552.10 ms
llama_print_timings:      sample time =      29.03 ms /    75 runs   (    0.39 ms per token,  2583.36 tokens per second)
llama_print_timings: prompt eval time =     881.53 ms /    24 tokens (   36.73 ms per token,    27.23 tokens per second)
llama_print_timings:        eval time =    3986.34 ms /    74 runs   (   53.87 ms per token,    18.56 tokens per second)
llama_print_timings:       total time =    5271.22 ms /    98 tokens



> Finished chain.


{'question': 'How can I love my husband more',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='\nAI: Hello! How can I assist you today?'),
  HumanMessage(content='How can I love my husband more'),
  AIMessage(content="?\nAI: There are many ways to love your husband more. Some ideas include expressing gratitude for him, showing affection through physical touch and words of encouragement, spending quality time together, and actively listening to his needs and concerns. It's also important to communicate openly and honestly with him about your feelings and to be supportive of his goals and aspirations.")],
 'text': "?\nAI: There are many ways to love your husband more. Some ideas include expressing gratitude for him, showing affection through physical touch and words of encouragement, spending quality time together, and actively listening to his needs and concerns. It's also important to communicate openly and honestly with him about your feelings and to be supportive o

In [6]:
conversation({"question": "it is a sunny day, should I make BBQ?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: Keep your answers very short, succinct, and to the point.
Human: hi
AI: 
AI: Hello! How can I assist you today?
Human: How can I love my husband more
AI: ?
AI: There are many ways to love your husband more. Some ideas include expressing gratitude for him, showing affection through physical touch and words of encouragement, spending quality time together, and actively listening to his needs and concerns. It's also important to communicate openly and honestly with him about your feelings and to be supportive of his goals and aspirations.
Human: it is a sunny day, should I make BBQ?


Llama.generate: prefix-match hit

llama_print_timings:        load time =    6552.10 ms
llama_print_timings:      sample time =      31.14 ms /    82 runs   (    0.38 ms per token,  2633.27 tokens per second)
llama_print_timings: prompt eval time =    5778.13 ms /    94 tokens (   61.47 ms per token,    16.27 tokens per second)
llama_print_timings:        eval time =    4416.99 ms /    81 runs   (   54.53 ms per token,    18.34 tokens per second)
llama_print_timings:       total time =   10714.68 ms /   175 tokens



> Finished chain.


{'question': 'it is a sunny day, should I make BBQ?',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='\nAI: Hello! How can I assist you today?'),
  HumanMessage(content='How can I love my husband more'),
  AIMessage(content="?\nAI: There are many ways to love your husband more. Some ideas include expressing gratitude for him, showing affection through physical touch and words of encouragement, spending quality time together, and actively listening to his needs and concerns. It's also important to communicate openly and honestly with him about your feelings and to be supportive of his goals and aspirations."),
  HumanMessage(content='it is a sunny day, should I make BBQ?'),
  AIMessage(content='\nAI: ?\nAI: Yes, it sounds like a great idea! Grilling can be a fun and delicious way to spend time outdoors on a sunny day. You could try making burgers, hot dogs, chicken, or even grilled vegetables for a healthy option. Just remember to keep an eye on the weather forecast a

In [7]:
conversation({"question": "how is weather in Höstel, germany?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: Keep your answers very short, succinct, and to the point.
Human: hi
AI: 
AI: Hello! How can I assist you today?
Human: How can I love my husband more
AI: ?
AI: There are many ways to love your husband more. Some ideas include expressing gratitude for him, showing affection through physical touch and words of encouragement, spending quality time together, and actively listening to his needs and concerns. It's also important to communicate openly and honestly with him about your feelings and to be supportive of his goals and aspirations.
Human: it is a sunny day, should I make BBQ?
AI: 
AI: ?
AI: Yes, it sounds like a great idea! Grilling can be a fun and delicious way to spend time outdoors on a sunny day. You could try making burgers, hot dogs, chicken, or even grilled vegetables for a healthy option. Just remember to keep an eye on the weather forecast and make sure you have enough food and drinks for everyone.
Human:

Llama.generate: prefix-match hit

llama_print_timings:        load time =    6552.10 ms
llama_print_timings:      sample time =      23.36 ms /    56 runs   (    0.42 ms per token,  2397.77 tokens per second)
llama_print_timings: prompt eval time =    2799.78 ms /    96 tokens (   29.16 ms per token,    34.29 tokens per second)
llama_print_timings:        eval time =    3055.13 ms /    56 runs   (   54.56 ms per token,    18.33 tokens per second)
llama_print_timings:       total time =    6233.13 ms /   152 tokens



> Finished chain.


{'question': 'how is weather in Höstel, germany?',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='\nAI: Hello! How can I assist you today?'),
  HumanMessage(content='How can I love my husband more'),
  AIMessage(content="?\nAI: There are many ways to love your husband more. Some ideas include expressing gratitude for him, showing affection through physical touch and words of encouragement, spending quality time together, and actively listening to his needs and concerns. It's also important to communicate openly and honestly with him about your feelings and to be supportive of his goals and aspirations."),
  HumanMessage(content='it is a sunny day, should I make BBQ?'),
  AIMessage(content='\nAI: ?\nAI: Yes, it sounds like a great idea! Grilling can be a fun and delicious way to spend time outdoors on a sunny day. You could try making burgers, hot dogs, chicken, or even grilled vegetables for a healthy option. Just remember to keep an eye on the weather forecast and 

---
## 4.&nbsp; Challenge 😀
1. Play around with writing new prompts.
  * Try having an empty prompt, what does it do to the output?
  * Try having a funny prompt.
  * Try having a long, precise prompt.
  * Try all different kinds of prompts.
2. Try out the different types of memory. What effect does this have on the type of conversations you have? Slower or faster? Better or worse accuracy? Forgetting things or super precise?
  * [ConversationBufferWindowMemory](https://python.langchain.com/docs/modules/memory/types/buffer_window) - Only a set number of k remembered - e.g. last 6 inputs
  * [ConversationSummaryMemory](https://python.langchain.com/docs/modules/memory/types/summary) - summarises the history to save space
  * [ConversationSummaryBufferMemory](https://python.langchain.com/docs/modules/memory/types/summary_buffer) - summarises and restricted by token length
3. Try different LLMs with different types of prompts and memory. Which combination works best for you? Why?

In [9]:
##empty prompt
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

### Prompt ###
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
          ' ' 
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

### Memory ###
# We `return_messages=True` to fit into the MessagesPlaceholder
# `"chat_history"` must align with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key = "chat_history",
                                  return_messages=True)

### Chain ###
conversation1 = LLMChain(llm = llm,
                        prompt = prompt,
                        verbose = True,
                        memory = memory)

In [10]:
conversation1({"question": "hi"})



> Entering new LLMChain chain...
Prompt after formatting:
System:  
Human: hi


Llama.generate: prefix-match hit

llama_print_timings:        load time =    6552.10 ms
llama_print_timings:      sample time =       1.17 ms /    13 runs   (    0.09 ms per token, 11149.23 tokens per second)
llama_print_timings: prompt eval time =     359.89 ms /     6 tokens (   59.98 ms per token,    16.67 tokens per second)
llama_print_timings:        eval time =     626.10 ms /    12 runs   (   52.17 ms per token,    19.17 tokens per second)
llama_print_timings:       total time =    1006.36 ms /    18 tokens



> Finished chain.


{'question': 'hi',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='\nAI: Hello! How can I assist you today?')],
 'text': '\nAI: Hello! How can I assist you today?'}

In [11]:
conversation1({"question": "How can I update my LinkedIn?"})



> Entering new LLMChain chain...
Prompt after formatting:
System:  
Human: hi
AI: 
AI: Hello! How can I assist you today?
Human: How can I update my LinkedIn?


Llama.generate: prefix-match hit

llama_print_timings:        load time =    6552.10 ms
llama_print_timings:      sample time =      49.78 ms /   112 runs   (    0.44 ms per token,  2249.90 tokens per second)
llama_print_timings: prompt eval time =     774.48 ms /    24 tokens (   32.27 ms per token,    30.99 tokens per second)
llama_print_timings:        eval time =    6027.00 ms /   112 runs   (   53.81 ms per token,    18.58 tokens per second)
llama_print_timings:       total time =    7480.69 ms /   136 tokens



> Finished chain.


{'question': 'How can I update my LinkedIn?',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='\nAI: Hello! How can I assist you today?'),
  HumanMessage(content='How can I update my LinkedIn?'),
  AIMessage(content='\nAI: To update your LinkedIn, you can follow these steps:\n1. Log in to your LinkedIn account.\n2. Click on the "Profile" tab.\n3. Make any necessary updates to your profile information, such as your name, job title, and summary.\n4. Upload a new profile picture if desired.\n5. Preview your changes to make sure everything looks correct.\n6. Click "Save" to update your LinkedIn profile.\nIs there anything else I can help you with?')],
 'text': '\nAI: To update your LinkedIn, you can follow these steps:\n1. Log in to your LinkedIn account.\n2. Click on the "Profile" tab.\n3. Make any necessary updates to your profile information, such as your name, job title, and summary.\n4. Upload a new profile picture if desired.\n5. Preview your changes to make sure ev

In [12]:
 conversation1({"question": "How Can I use it zo get data analyst job?"})



> Entering new LLMChain chain...
Prompt after formatting:
System:  
Human: hi
AI: 
AI: Hello! How can I assist you today?
Human: How can I update my LinkedIn?
AI: 
AI: To update your LinkedIn, you can follow these steps:
1. Log in to your LinkedIn account.
2. Click on the "Profile" tab.
3. Make any necessary updates to your profile information, such as your name, job title, and summary.
4. Upload a new profile picture if desired.
5. Preview your changes to make sure everything looks correct.
6. Click "Save" to update your LinkedIn profile.
Is there anything else I can help you with?
Human: How Can I use it zo get data analyst job?


Llama.generate: prefix-match hit

llama_print_timings:        load time =    6552.10 ms
llama_print_timings:      sample time =      13.90 ms /   158 runs   (    0.09 ms per token, 11367.72 tokens per second)
llama_print_timings: prompt eval time =    3670.70 ms /   127 tokens (   28.90 ms per token,    34.60 tokens per second)
llama_print_timings:        eval time =    8412.02 ms /   157 runs   (   53.58 ms per token,    18.66 tokens per second)
llama_print_timings:       total time =   12313.80 ms /   284 tokens



> Finished chain.


{'question': 'How Can I use it zo get data analyst job?',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='\nAI: Hello! How can I assist you today?'),
  HumanMessage(content='How can I update my LinkedIn?'),
  AIMessage(content='\nAI: To update your LinkedIn, you can follow these steps:\n1. Log in to your LinkedIn account.\n2. Click on the "Profile" tab.\n3. Make any necessary updates to your profile information, such as your name, job title, and summary.\n4. Upload a new profile picture if desired.\n5. Preview your changes to make sure everything looks correct.\n6. Click "Save" to update your LinkedIn profile.\nIs there anything else I can help you with?'),
  HumanMessage(content='How Can I use it zo get data analyst job?'),
  AIMessage(content="\nAI: \nAI: To use LinkedIn to find a data analyst job, you can follow these steps:\n1. Optimize your LinkedIn profile by highlighting your relevant skills and experience in data analysis.\n2. Connect with other professionals

In [14]:
 conversation1({"question": "Why having a mentor is important for this job roles"})



> Entering new LLMChain chain...
Prompt after formatting:
System:  
Human: hi
AI: 
AI: Hello! How can I assist you today?
Human: How can I update my LinkedIn?
AI: 
AI: To update your LinkedIn, you can follow these steps:
1. Log in to your LinkedIn account.
2. Click on the "Profile" tab.
3. Make any necessary updates to your profile information, such as your name, job title, and summary.
4. Upload a new profile picture if desired.
5. Preview your changes to make sure everything looks correct.
6. Click "Save" to update your LinkedIn profile.
Is there anything else I can help you with?
Human: How Can I use it zo get data analyst job?
AI: 
AI: 
AI: To use LinkedIn to find a data analyst job, you can follow these steps:
1. Optimize your LinkedIn profile by highlighting your relevant skills and experience in data analysis.
2. Connect with other professionals in the data analytics field and join relevant groups.
3. Use LinkedIn's job search feature to find open positions that match your qua

Llama.generate: prefix-match hit

llama_print_timings:        load time =    6552.10 ms
llama_print_timings:      sample time =      71.55 ms /   169 runs   (    0.42 ms per token,  2362.15 tokens per second)
llama_print_timings: prompt eval time =    5367.38 ms /     9 tokens (  596.38 ms per token,     1.68 tokens per second)
llama_print_timings:        eval time =    9355.92 ms /   168 runs   (   55.69 ms per token,    17.96 tokens per second)
llama_print_timings:       total time =   10615.14 ms /   177 tokens



> Finished chain.


{'question': 'Why having a mentor is important for this job roles',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='\nAI: Hello! How can I assist you today?'),
  HumanMessage(content='How can I update my LinkedIn?'),
  AIMessage(content='\nAI: To update your LinkedIn, you can follow these steps:\n1. Log in to your LinkedIn account.\n2. Click on the "Profile" tab.\n3. Make any necessary updates to your profile information, such as your name, job title, and summary.\n4. Upload a new profile picture if desired.\n5. Preview your changes to make sure everything looks correct.\n6. Click "Save" to update your LinkedIn profile.\nIs there anything else I can help you with?'),
  HumanMessage(content='How Can I use it zo get data analyst job?'),
  AIMessage(content="\nAI: \nAI: To use LinkedIn to find a data analyst job, you can follow these steps:\n1. Optimize your LinkedIn profile by highlighting your relevant skills and experience in data analysis.\n2. Connect with other pro